# sustainability_cluster_eda

In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.environment
sustainability_collection = db.sustainability

In [2]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from collections import Counter

from textblob import TextBlob

In [3]:
import pickle

In [24]:
with open('sustainability_clusters_v2.pickle','rb') as read_file:
    clusters2 = pickle.load(read_file)

In [51]:
clusters2

array([6, 2, 6, ..., 4, 2, 6], dtype=int32)

In [27]:
tweet_list = []
for tweet in sustainability_collection.aggregate([{'$match': {'lang': 'en'}}]):
    tweet_list.append(tweet['text'])

In [28]:
tweet_df = pd.DataFrame()

In [29]:
tweet_df = pd.DataFrame(tweet_list, columns = ['tweet'])    

In [30]:
tweet_df['clusters'] = clusters2

In [34]:
tweet_df['polarity'] = tweet_df.tweet.apply(lambda x: TextBlob(x).polarity)
tweet_df['subjectivity'] = tweet_df.tweet.apply(lambda x: TextBlob(x).subjectivity)

In [35]:
tweet_df.head()

,tweet,clusters,polarity,subjectivity
0,RT @angelurena: This visit concludes this week...,6,0.000000,0.00
1,Are you in Melbourne or Sydney? Join us at #SP...,2,0.500000,0.90
2,Why Vienna Does So Well on Quality-of-Life Ran...,6,0.000000,0.00
3,RT @APLShipping: APL achieved a 50.7% reductio...,6,0.000000,0.00
4,RT @AmazonWatch: “Hydroelectricity may appear ...,6,0.233333,0.85


In [36]:
clustered_tweets = tweet_df.filter(items = ['clusters', 'polarity', 'subjectivity'])

In [56]:
clustered_tweets.groupby(by = clusters).mean()[['polarity','subjectivity']]

,polarity,subjectivity
0,-0.000241,0.227166
1,-0.251852,0.262963
2,0.277126,0.432977
3,-0.100000,0.050000
4,0.303711,0.460179
5,0.500000,0.500000
6,-0.100000,0.200000
7,0.000000,0.000000
8,0.500000,0.500000
9,0.998322,0.898658


In [53]:
cluster_counts = Counter(clusters2)

for cluster, count in sorted(topic_counts.items()):
    print("Cluster Number:", cluster, "Percent:", round(count / sum(cluster_counts.values()),2))

Cluster Number: 0 Percent: 0.07
Cluster Number: 1 Percent: 0.01
Cluster Number: 2 Percent: 0.19
Cluster Number: 3 Percent: 0.04
Cluster Number: 4 Percent: 0.01
Cluster Number: 5 Percent: 0.03
Cluster Number: 6 Percent: 0.29
Cluster Number: 7 Percent: 0.04
Cluster Number: 8 Percent: 0.21
Cluster Number: 9 Percent: 0.01
Cluster Number: 10 Percent: 0.0
Cluster Number: 11 Percent: 0.01
Cluster Number: 12 Percent: 0.02
Cluster Number: 13 Percent: 0.02
Cluster Number: 14 Percent: 0.04
Cluster Number: 15 Percent: 0.01


In [43]:
with open ('tweet_df.pickle', 'wb') as to_write:
    pickle.dump(tweet_df, to_write)

In [44]:
with open ('clustered_tweets.pickle', 'wb') as to_write:
    pickle.dump(clustered_tweets, to_write)